In [1]:
## Design of Cyberphysical Systems - Project 

Shanmugadev Thiruselvam , Emrah Kichukov

# Paper Folding Processflow Simulation

The first step would be to implement the ressources along with its attributes and methods
### 1. Ressources


In [2]:
import simpy
import random

#Defining Resources (Paper  ,Robot , Operator ,Paper Loading Station)

#defining Paper 

class Paper:
    def __init__(self, env, workstation1):
        self.is_defective = False
        pass

class PaperLoadingstation (object):
    # Initally the paper laodingstation has 100 papers.
    def __init__(self,env):
        self.papers = simpy.PaperLoadingstation (env, init=100, capaciy = 100)

    def check_availability_paperloadingstation(self,operator):
        # check if there is enough paper on the Loading station

        # This takes between 1-3 seconds
         yield self.env.timeout(random.randint(1, 3))

         if self.paperloadingstation == 1:
           # if there are enough paper, return True
           # print("there are enough paper available ")
           return True
         else:
            print("There are not enough paper available on the station , please refill ")
            
            return False
         
    def pick_paper(self,operator,workstation1,Robot):
        while not self.env.process(self.check_availability_paperloadingstation(Robot)):
         yield self.env.timeout(1)
         yield self.pickup_paperloadingstation.get(1) & self.env.process(self.restock())  # & self.env.process(workstation3.pickup_paper(robot))
    
    def restock_paperloadingstation(self):
         #restock the paperdispenser , this takes between 15-25 seconds
          yield self.env.timeout(random.randint(5,10))
          #restock the paper dispenser
          yield self.pickup_paperdispenser.put(100)
          print("The paper dispenser has been restocked")

#defining Robot
    
class Robot(object):
   
    def __init__(self,env,workstation1):
        self.env = env
        
        self.workstation1 = workstation1
        self.robot = simpy.Resource(env,capacity= 1)
        

    #The calibration takes place as a first step, this will takes around 2-3 Seconds
    def actuate(self,workstation1):
        yield self.env.timeout(random.randint(2,3))
        
    #the paper is loaded into the robot
    def load_paper(self,paper,paperloadingstation):
        yield self.env.timeout(random.randint(2,3))
        
     #the paper is loaded into Folding Base by the  robot , where the folding operation takes place
    def load_paper_to_foldingbase(self,paper):
        yield self.env.timeout(random.randint(2,3)) 
        
    
    #the Folding operation takes place by the robot
    def fold_paper(self,paper):
        yield self.env.timeout(random.randint(4,7))
        
    #the paper is unloaded from the Folding Base by the robot
    def unload_paper_from_foldingbase(self,paper):
        yield self.env.timeout(random.randint(2,3))
        
    def return_to_start(self):
        yield self.env.timeout(random.randint(2, 3))
    
    #At the end ,hand over the folded paper to the next station for further operations
    def hand_over_paper(self,workstation2):
        yield self.env.timeout (random.randint(2,3))
        

#defining Operator
        

class Operator (object):
    def __init__(self,env,workstation1,folded_papers):
        self.env = env
        self.workstation1 = workstation1
        self.folded_papers = folded_papers
        
        self.quality_check_times = {'check1': []}
        self.total_idle_time = 0
        
        
    def adjust_paper_loading(self, paper):
        # Check if the paper is loaded properly
        if random.random() < 0.3:  # 30% chance of loading error
            print("Paper not loaded properly. Adjusting...")
            yield self.env.timeout(random.randint(1, 3))
            print("Adjustment completed.")
        else:
            print("Paper is loaded properly.")
        
        

    def quality_check_fold(self, paper):
        # Quality check after Folding Operation
        start_time = self.env.now
        yield self.env.timeout(random.randint(2, 3))
        end_time = self.env.now
        self.quality_check_times['check1'].append(end_time - start_time)

        # Assume a 10% chance of a defect
        if random.random() < 0.1:
            paper.is_defective = True

        if paper.is_defective:
            # If the paper is defective, send it to waste
            print("Folded Paper has a defect and goes to waste.")
            yield self.env.process(self.workstation1.WasteStation.dispose(paper))
            robot = Robot(self.env, self.workstation1)
            yield self.env.process(self.workstation1.workstation1_process(self, Paper(self.env, self.workstation1), robot))
            
        else:
            print("Folded Paper passed quality check.")
        
        
        self.idle_start_time = self.env.now
        
    def align_paper(self, paper):
        # Check if the paper is aligned properly
        if random.random() < 0.3:  # 30% chance of misalignment
            print("Paper misaligned. Aligning...")
            yield self.env.timeout(random.randint(1, 3))
            print("Alignment completed.")
        else:
            print("Paper is aligned properly.")
    
        

#defining Workstation2 
class Workstation2(object):
    def __init__(self,env):
        self.env = env
        
class WasteStation(object):
    def __init__(self, env):
        self.env = env

    def dispose(self,paper):
        # Waste disposal process
        yield self.env.timeout(random.randint(2, 3))
        print("Paper has been disposed of as waste.")
         




In [4]:
#Defining the workstatiion1

class WorkStation1(object):
    def __init__(self,env,paper_list,robot,Workstation2,operator,folded_papers):
        self.env = env
        self.folded_papers = folded_papers
        
        self.operator = operator(env,self, self.folded_papers )
        self.idle_start_time = 0
        
        self.robot = robot(env,self)
        self.workstation2 = Workstation2(env)
        
        self.WasteStation = WasteStation(env) 
         
        self.folded_papers = 0
        self.total_operation_time = 0
        self.totalfolding1_time = 0
        self.paper_loading_verification_time = 0
        self.paper_alignment_verification_time = 0
        self.totalfolding2_time = 0
        self.total_quality_check_time = 0
        self.defective_papers = 0
        self.operator_idle_time = 0
        self.start_idle_time = 0
        self.total_idle_time = 0
        self.production_rate_data = []
        self.operation_times = []